In [1]:
import pandas as pd
import numpy as np 
from math import floor

In [2]:
df = pd.read_excel('F:\Output Messenger\Received Files\DetailOutage(2020-03-20)_2G-3G-4G.xlsx')
df['StartOfOutage'] = pd.to_datetime(df['StartOfOutage'])
df['EndtOfOutage'] = pd.to_datetime(df['EndOfOutage'])

df.loc[(df['NE'].str[-1] == 'E') & (~df['CellName'].str[3].isin(['L','G','U'])) ,'Site'] = \
df.loc[(df['NE'].str[-1] == 'E') & (~df['CellName'].str[3].isin(['L','G','U'])),'CellName'].str[:6]

df.loc[(df['NE'].str[-1] != 'E') | (df['CellName'].str[3].isin(['L','G','U'])),'Site'] = \
df.loc[(df['NE'].str[-1] != 'E') | (df['CellName'].str[3].isin(['L','G','U'])),'CellName'].str[:8]

df.rename(columns={'Technology': 'Tech'}, inplace=True)

df.loc[df['Tech'] == '4G','NE'] = df.loc[df['Tech'] == '4G','NE'] + '_LTE'

df.rename(columns={'NE': 'BSC-RNC'}, inplace=True)

#df.rename(columns={'CellName': 'Cell', 'NE': 'BSC-RNC'}, inplace = True)
df.sort_values(by = ['BSC-RNC','StartOfOutage','Down_Time(Seconds)'], inplace = True)
#print(type(df['StartOfOutage'][0]))
df2 = df[['StartOfOutage','Down_Time(Seconds)']].diff()#.dt.total_seconds()/3600
df['diff'] = (df['BSC-RNC'] != df['BSC-RNC'].shift(1).fillna(df['BSC-RNC'])).astype(int)
df2.columns = ['d_st_hour','d_duration']
df = pd.concat([df, df2], axis = 1)

df['d_st_hour'] = df['d_st_hour'].dt.total_seconds()/3600
df['d_duration'] = df['d_duration']/3600

df['change_flag'] = 0
df[['d_st_hour','d_duration']] = df[['d_st_hour','d_duration']].fillna(0) 

flag_old_grouping = False
if flag_old_grouping:
    df.loc[(df['d_st_hour'] != 0) | (df['d_duration'].abs()> np.minimum(1/6, 0.1 * df['Down_Time(Seconds)']/3600)),'change_flag'] = 1
else: # new grouping
    df.loc[(df['d_st_hour'] != 0) | (df['d_duration'].abs()> \
                                     np.minimum(1/3, np.maximum(0.1,(4/75 + 7/25* df['Down_Time(Seconds)']/3600)) * df['Down_Time(Seconds)']/3600)),'change_flag'] = 1
#df_all['cs'] = df_all['change_flag'].cumsum()

#df_all
df['no'] = df.groupby(['BSC-RNC'])['change_flag'].cumsum()
print(df.head())
df2 = df.copy()
tdf = df.groupby(['BSC-RNC','no'])['Site'].nunique()#transform('nunique')
tdf = tdf.reset_index()
print(tdf.head())
#tdf.to_csv('wao_22.csv', index = False, encoding = 'utf-8-sig')
# merge on no and BSC-RNC
df = df.merge(tdf, left_on = ['BSC-RNC','no'], right_on = ['BSC-RNC','no'])
##print(df[df['BSC-RNC'] == 'B069N'])
#df.to_csv('test.csv', index = False, encoding = 'utf-8-sig')

print('merging ...')

dfc = pd.read_excel('F:\Output Messenger\Received Files\BSCRNC_Sites.xlsx')
dfc.loc[dfc['Tech']=='4G','BSC-RNC'] = dfc.loc[dfc['Tech']=='4G','BSC-RNC'] + '_LTE'
df = df.merge(dfc, left_on = ['BSC-RNC','Tech'], right_on = ['BSC-RNC','Tech'])

#aras = pd.read_excel('..//Data//Aras.xlsx')
#df = df.merge(aras[['Location','Latitude','Longitude']], on = 'Location', how = 'left')

#df.drop(['Latitude_x','Longitude_x'], axis = 1, inplace = True)

#dff = df[df['BSC-RNC'] == 'B069N']
#dff.to_csv('test.csv', index = False, encoding = 'utf-8-sig')
print(df.columns)

df['outage_percent'] = df['Site_y']/df['Site']
df = df[df['outage_percent'] > 0.3]
#df = df[df['Tech']=='4G']
df.rename(columns={'CellName': 'Cell'}, inplace = True)

#df.to_csv('wao_tickets_included.csv', index = False, encoding = 'utf-8-sig')

df.to_csv('wao_16March.csv', index = False, encoding = 'utf-8-sig')

         CellName Province Index      BSC-RNC Location StartOfOutage  \
10534  KZ2U0209C2             KZ  Ahwaz_R611H   KZ0209    2020-03-20   
12005  KZ2U0209B1             KZ  Ahwaz_R611H   KZ0209    2020-03-20   
11489  KZ2U0209C1             KZ  Ahwaz_R611H   KZ0209    2020-03-20   
8794   KZ2U0209B2             KZ  Ahwaz_R611H   KZ0209    2020-03-20   
9337   KZ2U0209A2             KZ  Ahwaz_R611H   KZ0209    2020-03-20   

      EndOfOutage  Hour  Down_Time(Seconds) Tech  BCCH_DownTime_Duration(s)  \
10534  2020-03-20     1                35.0   3G                        NaN   
12005  2020-03-20     1                45.0   3G                        NaN   
11489  2020-03-20     1                60.0   3G                        NaN   
8794   2020-03-20     1                69.0   3G                        NaN   
9337   2020-03-20     1                69.0   3G                        NaN   

      EndtOfOutage      Site  diff  d_st_hour  d_duration  change_flag  no  
10534   2020-03